In [31]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader


In [32]:
transform=transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    #transforms.Normalize((0.1307,),(0.3081,))
    transforms.Normalize(
        mean=[0.485,0.456,0.406],
        std=[0.229,0.224,0.225]
    )
])

fulldata=datasets.EuroSAT(
    download=True,
    root='./data',
    transform=transform
)

from torch.utils.data import Subset


train_classes = [0, 1, 2, 3, 4, 5, 6, 7]
test_classes = [8, 9]


def filter_dataset_by_class(dataset, class_list):
    indices = [i for i, target in enumerate(dataset.targets) if target in class_list]
    return Subset(dataset, indices)


data = filter_dataset_by_class(fulldata, train_classes)
test_data = filter_dataset_by_class(fulldata, test_classes)

In [33]:
fulldata

Dataset EuroSAT
    Number of datapoints: 27000
    Root location: ./data
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

In [34]:
import torchvision.models as models

In [35]:
vgg=models.vgg16(pretrained=True)

/home/stark/Deep Learning/temp.env/lib/python3.13/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [36]:
class VGGEmbedding(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = vgg.features
        self.avgpool = vgg.avgpool
        
        self.classifier = nn.Sequential(*list(vgg.classifier.children())[:-1])
    
    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x
    


In [37]:
from torchvision.models import resnet18

class ResNet18Embedding(nn.Module):
    def __init__(self):
        super().__init__()
        resnet = resnet18(pretrained=True)
        self.features = nn.Sequential(*list(resnet.children())[:-1]) # Remove final FC layer
        self.embedding = nn.Linear(512, 10)  

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.embedding(x)
        return x  # Output: 512-dimensional

In [ ]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = ResNet18Embedding().to(device)

# # Freeze base layers
# for param in model.features.parameters():
#     param.requires_grad = False

# # Show what’s frozen vs trainable
# print("\n=== Layer-wise requires_grad ===")
# for name, param in model.named_parameters():
#     status = "✅ Trainable" if param.requires_grad else "❌ Frozen"
#     print(f"{name}: {status}")


# # Summary (if needed)
# from torchsummary import summary
# summary(model, input_size=(3, 224, 224))



=== Layer-wise requires_grad ===
features.0.weight: ❌ Frozen
features.1.weight: ❌ Frozen
features.1.bias: ❌ Frozen
features.4.0.conv1.weight: ❌ Frozen
features.4.0.bn1.weight: ❌ Frozen
features.4.0.bn1.bias: ❌ Frozen
features.4.0.conv2.weight: ❌ Frozen
features.4.0.bn2.weight: ❌ Frozen
features.4.0.bn2.bias: ❌ Frozen
features.4.1.conv1.weight: ❌ Frozen
features.4.1.bn1.weight: ❌ Frozen
features.4.1.bn1.bias: ❌ Frozen
features.4.1.conv2.weight: ❌ Frozen
features.4.1.bn2.weight: ❌ Frozen
features.4.1.bn2.bias: ❌ Frozen
features.5.0.conv1.weight: ❌ Frozen
features.5.0.bn1.weight: ❌ Frozen
features.5.0.bn1.bias: ❌ Frozen
features.5.0.conv2.weight: ❌ Frozen
features.5.0.bn2.weight: ❌ Frozen
features.5.0.bn2.bias: ❌ Frozen
features.5.0.downsample.0.weight: ❌ Frozen
features.5.0.downsample.1.weight: ❌ Frozen
features.5.0.downsample.1.bias: ❌ Frozen
features.5.1.conv1.weight: ❌ Frozen
features.5.1.bn1.weight: ❌ Frozen
features.5.1.bn1.bias: ❌ Frozen
features.5.1.conv2.weight: ❌ Frozen
feature

In [22]:
from torchvision.models import resnet50

class ResNet50Embedding(nn.Module):
    def __init__(self):
        super().__init__()
        resnet = resnet50(pretrained=True)
        self.features = nn.Sequential(*list(resnet.children())[:-1])  # Remove final FC layer
        self.embedding = nn.Linear(512, 10)

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.embedding(x)
        return x  # Output: 2048-dimensional


In [29]:
from torchvision.models import densenet121

class DenseNet121Embedding(nn.Module):
    def __init__(self):
        super().__init__()
        densenet = densenet121(pretrained=True)
        self.features = densenet.features  # Convolutional backbone
        self.pool = nn.AdaptiveAvgPool2d((1, 1))  # Match ResNet pooling
        self.embedding = nn.Linear(1024, 10)

    def forward(self, x):
        x = self.features(x)
        x = self.pool(x)
        x = torch.flatten(x, 1)
        x = self.embedding(x)
        return x  # Output: 1024-dimensional


In [80]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.conv1=nn.Conv2d(3,16,kernel_size=3,padding=1)
        self.conv2=nn.Conv2d(16,32,kernel_size=3,padding=1)
        self.conv3=nn.Conv2d(32,64,kernel_size=3,padding=1)
        self.conv4=nn.Conv2d(64,128,kernel_size=3,padding=1)

        self.flat=nn.Flatten()

        self.pool=nn.MaxPool2d(2,2)

        self.ReLU=nn.ReLU()

        self.full=nn.Linear(128*4*4,512)
    def forward(self,x):
        x=self.conv1(x)
        x=self.ReLU(x)
        x=self.pool(x)

        x=self.conv2(x)
        x=self.ReLU(x)
        x=self.pool(x)

        x=self.conv3(x)
        x=self.ReLU(x)
        x=self.pool(x)

        x=self.conv4(x)
        x=self.ReLU(x)
        x=self.pool(x)

        x=self.flat(x)

        final=self.full(x)

        return final



In [81]:
from torch.utils.data import Dataset
class fewshot_dataset(Dataset):
    def __init__(self,data,way,shot,query,episodes):
        self.data=data
        self.way=way
        self.shot=shot
        self.query=query
        self.episodes=episodes

        self.class_to_indices=self._build_class_index()
    
    def _build_class_index(self):
        class_index={}
        for idx, (x,y) in enumerate(self.data):
            if y not in class_index:
                class_index[y]=[]
            class_index[y].append(idx)
        return class_index
    
    def __len__(self):
        return self.episodes
    
    def __getitem__(self,idx):
        selected_classes=random.sample(list(self.class_to_indices.keys()), self.way)

        support_images=[]
        support_labels=[]
        query_images=[]
        query_labels=[]


        label_map={class_name: i for i, class_name in enumerate(selected_classes)}
        
        for class_name in selected_classes:
            all_indices=self.class_to_indices[class_name]
            selected_indices=random.sample(all_indices,self.shot+self.query)

            support_idx=selected_indices[:self.shot]
            query_idx=selected_indices[self.shot:]

            for idx in support_idx:
                image,_=self.data[idx]
                support_images.append(image)
                support_labels.append(torch.tensor(label_map[class_name]))
            
            for idx in query_idx:
                image,_=self.data[idx]
                query_images.append(image)
                query_labels.append(torch.tensor(label_map[class_name]))
        return(
             torch.stack(support_images),
             torch.stack(support_labels),
             torch.stack(query_images),
             torch.stack(query_labels)
         )

In [82]:
def compute_prototypes(support_embeddings,support_labels,way):
    embedding_dimensions=support_embeddings.size(-1)
    prototypes=torch.zeros(way,embedding_dimensions).to(support_embeddings.device)

    for c in range(way):
        class_mask=(support_labels==c)
        class_embeddings=support_embeddings[class_mask]
        prototypes[c]=class_embeddings.mean(dim=0)
    return prototypes

def classify_queries(prototypes,query_embeddings):
    n_query=query_embeddings.size(0)
    way=prototypes.size(0)
    
    query_exp=query_embeddings.unsqueeze(1).expand(n_query,way,-1)  ##important
    prototypes_exp=prototypes.unsqueeze(0).expand(n_query,way,-1)  ##impotant

    distances=torch.sum((query_exp-prototypes_exp)**2,dim=2)

    logits=-distances
    return logits



In [83]:
few_dataset=fewshot_dataset(
    data=data,
    way=2,
    shot=1,
    query=2,
    episodes=100
)

dataloader=DataLoader(few_dataset,batch_size=1,shuffle=True)

In [84]:
import random
import torch.optim as optim


In [89]:
#model=CNN()
model=VGGEmbedding()
##Need to ask
for param in model.features.parameters():
    param.requires_grad=False
model.classifier[3]=nn.Linear(model.classifier[3].in_features,10)
device=torch.device("cuda")
model=model.to(device)

optimizer=optim.Adam(model.parameters(),lr=1e-4)
loss_fn=nn.CrossEntropyLoss()


epochs=20

In [ ]:
# torch.cuda.empty_cache()

In [ ]:
for epoch in range(epochs):
    model.train()
    total_loss=0
    total_correct=0
    total_queries=0

    for episode in dataloader:
        support_images,support_labels, query_images, query_labels=episode

        support_images=(support_images.squeeze(0)).to(device)
        query_images=(query_images.squeeze(0)).to(device)
        support_labels=(support_labels.view(-1)).to(device)
        query_labels=(query_labels.view(-1)).to(device)

        support_embeddings=model(support_images)
        query_embeddings=model(query_images)

        n_way=torch.unique(support_labels).size(0)
        prototypes=compute_prototypes(support_embeddings,support_labels,n_way)

        logits=classify_queries(prototypes, query_embeddings)
        loss=loss_fn(logits, query_labels)
        total_loss+=loss.item()

        preds=torch.argmax(logits,dim=1)
        correct=(preds==query_labels).sum().item()
        total_correct+=correct
        total_queries+=query_labels.size(0)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # del support_images, query_images, support_labels, query_labels
        # del support_embeddings, query_embeddings, prototypes, logits, preds, loss
        # torch.cuda.empty_cache()
        # import gc
        # gc.collect()

    avg_loss=total_loss/len(dataloader)
    acc=total_correct/total_queries *100

    print(f"Epoch:[{epoch+1}/{epochs}] --> Loss:{avg_loss} and Accuracy:{acc}")

Epoch:[1/20] --> Loss:1.2065574595332145 and Accuracy:50.5
Epoch:[2/20] --> Loss:0.6932703733444214 and Accuracy:50.0
Epoch:[3/20] --> Loss:0.6931528782844544 and Accuracy:50.0
Epoch:[4/20] --> Loss:0.6931471824645996 and Accuracy:50.0
Epoch:[5/20] --> Loss:0.6931471884250641 and Accuracy:50.0
Epoch:[6/20] --> Loss:0.6934351646900176 and Accuracy:50.0
Epoch:[7/20] --> Loss:0.6931644684076309 and Accuracy:50.0
Epoch:[8/20] --> Loss:0.693147297501564 and Accuracy:50.0
Epoch:[9/20] --> Loss:0.6931471824645996 and Accuracy:50.0
Epoch:[10/20] --> Loss:0.6931471824645996 and Accuracy:50.0
Epoch:[11/20] --> Loss:0.6931471824645996 and Accuracy:50.0
Epoch:[12/20] --> Loss:0.6931471824645996 and Accuracy:50.0
Epoch:[13/20] --> Loss:0.6931471824645996 and Accuracy:50.0
Epoch:[14/20] --> Loss:0.6932056248188019 and Accuracy:50.0
Epoch:[15/20] --> Loss:0.6931471824645996 and Accuracy:50.0
Epoch:[16/20] --> Loss:0.6931471824645996 and Accuracy:50.0
Epoch:[17/20] --> Loss:0.6931471824645996 and Accu

AttributeError: 'VGGEmbedding' object has no attribute 'summary'

In [67]:
few_dataset=fewshot_dataset(
    data=test_data,
    way=2,
    shot=1,
    query=2,
    episodes=100
)

test_loader=DataLoader(few_dataset,batch_size=1,shuffle=True)

In [ ]:
model.eval()
total_correct = 0
total_queries = 0

with torch.no_grad():
    for episode in test_loader:
        support_images, support_labels, query_images, query_labels = episode

        support_images = support_images.squeeze(0).to(device)
        query_images = query_images.squeeze(0).to(device)
        support_labels = support_labels.view(-1).to(device)
        query_labels = query_labels.view(-1).to(device)

        support_embeddings = model(support_images)
        query_embeddings = model(query_images)

        n_way = torch.unique(support_labels).size(0)
        prototypes = compute_prototypes(support_embeddings, support_labels, n_way)

        logits = classify_queries(prototypes, query_embeddings)
        preds = torch.argmax(logits, dim=1)

        total_correct += (preds == query_labels).sum().item()
        total_queries += query_labels.size(0)

acc = total_correct / total_queries * 100
print(f"1-shot Accuracy on Unseen Classes (8 & 9) on VGGNET: {acc:.2f}%")


1-shot Accuracy on Unseen Classes (8 & 9): 50.00%


In [ ]:

#model=CNN()
model=ResNet18Embedding()
##Need to ask
for param in model.features.parameters():
    param.requires_grad=False
device=torch.device("cuda")
model=model.to(device)

optimizer=optim.Adam(model.parameters(),lr=1e-4)
loss_fn=nn.CrossEntropyLoss()

epochs=20

torch.cuda.empty_cache()

for epoch in range(epochs):
    model.train()
    total_loss=0
    total_correct=0
    total_queries=0

    for episode in dataloader:
        support_images,support_labels, query_images, query_labels=episode

        support_images=(support_images.squeeze(0)).to(device)
        query_images=(query_images.squeeze(0)).to(device)
        support_labels=(support_labels.view(-1)).to(device)
        query_labels=(query_labels.view(-1)).to(device)

        support_embeddings=model(support_images)
        query_embeddings=model(query_images)

        n_way=torch.unique(support_labels).size(0)
        prototypes=compute_prototypes(support_embeddings,support_labels,n_way)

        logits=classify_queries(prototypes, query_embeddings)
        loss=loss_fn(logits, query_labels)
        total_loss+=loss.item()

        preds=torch.argmax(logits,dim=1)
        correct=(preds==query_labels).sum().item()
        total_correct+=correct
        total_queries+=query_labels.size(0)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # del support_images, query_images, support_labels, query_labels
        # del support_embeddings, query_embeddings, prototypes, logits, preds, loss
        # torch.cuda.empty_cache()
        # import gc
        # gc.collect()

    avg_loss=total_loss/len(dataloader)
    acc=total_correct/total_queries *100

    print(f"Epoch:[{epoch+1}/{epochs}] --> Loss:{avg_loss} and Accuracy:{acc}")

few_dataset=fewshot_dataset(
    data=test_data,
    way=2,
    shot=1,
    query=2,
    episodes=100
)

test_loader=DataLoader(few_dataset,batch_size=1,shuffle=True)

model.eval()
total_correct = 0
total_queries = 0

with torch.no_grad():
    for episode in test_loader:
        support_images, support_labels, query_images, query_labels = episode

        support_images = support_images.squeeze(0).to(device)
        query_images = query_images.squeeze(0).to(device)
        support_labels = support_labels.view(-1).to(device)
        query_labels = query_labels.view(-1).to(device)

        support_embeddings = model(support_images)
        query_embeddings = model(query_images)

        n_way = torch.unique(support_labels).size(0)
        prototypes = compute_prototypes(support_embeddings, support_labels, n_way)

        logits = classify_queries(prototypes, query_embeddings)
        preds = torch.argmax(logits, dim=1)

        total_correct += (preds == query_labels).sum().item()
        total_queries += query_labels.size(0)

acc = total_correct / total_queries * 100
print(f"1-shot Accuracy on Unseen Classes (8 & 9) on Resnet18: {acc:.2f}%")


NameError: name 'a' is not defined

In [ ]:
#model=CNN()
model=ResNet50Embedding()
##Need to ask
for param in model.features.parameters():
    param.requires_grad=False
device=torch.device("cuda")
model=model.to(device)

optimizer=optim.Adam(model.parameters(),lr=1e-4)
loss_fn=nn.CrossEntropyLoss()

epochs=20

torch.cuda.empty_cache()

for epoch in range(epochs):
    model.train()
    total_loss=0
    total_correct=0
    total_queries=0

    for episode in dataloader:
        support_images,support_labels, query_images, query_labels=episode

        support_images=(support_images.squeeze(0)).to(device)
        query_images=(query_images.squeeze(0)).to(device)
        support_labels=(support_labels.view(-1)).to(device)
        query_labels=(query_labels.view(-1)).to(device)

        support_embeddings=model(support_images)
        query_embeddings=model(query_images)

        n_way=torch.unique(support_labels).size(0)
        prototypes=compute_prototypes(support_embeddings,support_labels,n_way)

        logits=classify_queries(prototypes, query_embeddings)
        loss=loss_fn(logits, query_labels)
        total_loss+=loss.item()

        preds=torch.argmax(logits,dim=1)
        correct=(preds==query_labels).sum().item()
        total_correct+=correct
        total_queries+=query_labels.size(0)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # del support_images, query_images, support_labels, query_labels
        # del support_embeddings, query_embeddings, prototypes, logits, preds, loss
        # torch.cuda.empty_cache()
        # import gc
        # gc.collect()

    avg_loss=total_loss/len(dataloader)
    acc=total_correct/total_queries *100

    print(f"Epoch:[{epoch+1}/{epochs}] --> Loss:{avg_loss} and Accuracy:{acc}")

few_dataset=fewshot_dataset(
    data=test_data,
    way=2,
    shot=1,
    query=2,
    episodes=100
)

test_loader=DataLoader(few_dataset,batch_size=1,shuffle=True)

model.eval()
total_correct = 0
total_queries = 0

with torch.no_grad():
    for episode in test_loader:
        support_images, support_labels, query_images, query_labels = episode

        support_images = support_images.squeeze(0).to(device)
        query_images = query_images.squeeze(0).to(device)
        support_labels = support_labels.view(-1).to(device)
        query_labels = query_labels.view(-1).to(device)

        support_embeddings = model(support_images)
        query_embeddings = model(query_images)

        n_way = torch.unique(support_labels).size(0)
        prototypes = compute_prototypes(support_embeddings, support_labels, n_way)

        logits = classify_queries(prototypes, query_embeddings)
        preds = torch.argmax(logits, dim=1)

        total_correct += (preds == query_labels).sum().item()
        total_queries += query_labels.size(0)

acc = total_correct / total_queries * 100
print(f"1-shot Accuracy on Unseen Classes (8 & 9) on Resnet50: {acc:.2f}%")


In [ ]:
#model=CNN()
model=DenseNet121Embedding()
##Need to ask

device=torch.device("cuda")
model=model.to(device)

optimizer=optim.Adam(model.parameters(),lr=1e-4)
loss_fn=nn.CrossEntropyLoss()

epochs=20

torch.cuda.empty_cache()

for epoch in range(epochs):
    model.train()
    total_loss=0
    total_correct=0
    total_queries=0

    for episode in dataloader:
        support_images,support_labels, query_images, query_labels=episode

        support_images=(support_images.squeeze(0)).to(device)
        query_images=(query_images.squeeze(0)).to(device)
        support_labels=(support_labels.view(-1)).to(device)
        query_labels=(query_labels.view(-1)).to(device)

        support_embeddings=model(support_images)
        query_embeddings=model(query_images)

        n_way=torch.unique(support_labels).size(0)
        prototypes=compute_prototypes(support_embeddings,support_labels,n_way)

        logits=classify_queries(prototypes, query_embeddings)
        loss=loss_fn(logits, query_labels)
        total_loss+=loss.item()

        preds=torch.argmax(logits,dim=1)
        correct=(preds==query_labels).sum().item()
        total_correct+=correct
        total_queries+=query_labels.size(0)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # del support_images, query_images, support_labels, query_labels
        # del support_embeddings, query_embeddings, prototypes, logits, preds, loss
        # torch.cuda.empty_cache()
        # import gc
        # gc.collect()

    avg_loss=total_loss/len(dataloader)
    acc=total_correct/total_queries *100

    print(f"Epoch:[{epoch+1}/{epochs}] --> Loss:{avg_loss} and Accuracy:{acc}")

few_dataset=fewshot_dataset(
    data=test_data,
    way=2,
    shot=1,
    query=2,
    episodes=100
)

test_loader=DataLoader(few_dataset,batch_size=1,shuffle=True)

model.eval()
total_correct = 0
total_queries = 0

with torch.no_grad():
    for episode in test_loader:
        support_images, support_labels, query_images, query_labels = episode

        support_images = support_images.squeeze(0).to(device)
        query_images = query_images.squeeze(0).to(device)
        support_labels = support_labels.view(-1).to(device)
        query_labels = query_labels.view(-1).to(device)

        support_embeddings = model(support_images)
        query_embeddings = model(query_images)

        n_way = torch.unique(support_labels).size(0)
        prototypes = compute_prototypes(support_embeddings, support_labels, n_way)

        logits = classify_queries(prototypes, query_embeddings)
        preds = torch.argmax(logits, dim=1)

        total_correct += (preds == query_labels).sum().item()
        total_queries += query_labels.size(0)

acc = total_correct / total_queries * 100
print(f"1-shot Accuracy on Unseen Classes (8 & 9) on DenseNet121: {acc:.2f}%")
